In [2]:
import numpy as np
import pandas as pd
import json
import seaborn as sns
import matplotlib.pyplot as plt
from ieeg.auth import Session
from scipy import signal as sig
import os
from os.path import join as ospj
from os.path import exists as ospe
import pathlib
from tqdm import tqdm

In [2]:
import sys
sys.path.append('/users/wojemann/iEEG_processing')
from pioneer import Pioneer

In [3]:
with open('config.json','r') as f:
    config = json.load(f)

In [4]:
ptList = config['patients']

In [5]:
display(config['paths'])
raw_datapath = config['paths']['RAW_DATA']
usr = config['paths']['iEEG_USR']
pwd = config['paths']['iEEG_PWD']
ignore_list = config['ignore']

{'RAW_DATA': '/users/wojemann/stim-seizures/RAW_DATA',
 'iEEG_USR': 'wojemann',
 'iEEG_PWD': '/users/wojemann/woj_ieeglogin.bin'}

In [7]:
# wagon = Pioneer(usr,pwd,ptList[0])
# wagon.pull_annotations()
# wagon.filter_seizure_annotations()
# test = wagon.seizure_annotations

In [8]:
ptList

['HUP224_phaseII',
 'HUP225_phaseII',
 'HUP229_phaseII',
 'HUP230_phaseII',
 'HUP235_phaseII',
 'HUP238_phaseII',
 'HUP246_phaseII',
 'HUP247_phaseII',
 'HUP249_phaseII',
 'HUP250_phaseII',
 'HUP253_phaseII',
 'HUP224_CCEP',
 'HUP225_CCEP',
 'HUP229_CCEP',
 'HUP230_CCEP',
 'HUP235_CCEP',
 'HUP238_CCEP',
 'HUP246_CCEP',
 'HUP247_CCEP',
 'HUP249_CCEP',
 'HUP250_CCEP',
 'HUP253_CCEP']

In [12]:
for i,pt in enumerate(ptList[-9:]):
    save_path = ospj(raw_datapath,pt[:6])
    print(pt,save_path)
    if not ospe(save_path):
        os.makedirs(save_path)
    wagon = Pioneer(usr,pwd,pt)
    wagon.pull_annotations()
    wagon.filter_seizure_annotations()
    # wagon.seizure_annotations.loc[:,'ID'] = wagon.ID
    wagon.seizure_annotations.to_csv(ospj(save_path,f'seizure_annotations_{pt}.csv'))

HUP229_CCEP /users/wojemann/stim-seizures/RAW_DATA/HUP229
got 36 annotations on call # 1 covering 669795898 usec to 1239678711 usec
Filtered      10.00% of all annotations
HUP230_CCEP /users/wojemann/stim-seizures/RAW_DATA/HUP230
got 100 annotations on call # 1 covering 1470541015 usec to 3020485351 usec
got 100 annotations on call # 2 covering 3027179687 usec to 4424435546 usec
got 56 annotations on call # 3 covering 4429115234 usec to 5148346679 usec
Filtered       9.96% of all annotations
HUP235_CCEP /users/wojemann/stim-seizures/RAW_DATA/HUP235


/users/wojemann/iEEG_processing/pioneer.py:199: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_annotations['IEEG Time'] = t_sec


got 100 annotations on call # 1 covering 6366798828 usec to 8178477539 usec
got 100 annotations on call # 2 covering 8215919921 usec to 9400696289 usec
got 54 annotations on call # 3 covering 9437391601 usec to 10195240234 usec
Filtered       9.92% of all annotations
HUP238_CCEP /users/wojemann/stim-seizures/RAW_DATA/HUP238
got 100 annotations on call # 1 covering 6457686523 usec to 8513380859 usec
got 100 annotations on call # 2 covering 8514391601 usec to 9705616211 usec
got 100 annotations on call # 3 covering 9711195312 usec to 10702934570 usec
got 11 annotations on call # 4 covering 10706912109 usec to 10956357421 usec
Filtered       9.68% of all annotations
HUP246_CCEP /users/wojemann/stim-seizures/RAW_DATA/HUP246
got 100 annotations on call # 1 covering 10564452148 usec to 12222125000 usec
got 100 annotations on call # 2 covering 12255099609 usec to 13466364257 usec
got 100 annotations on call # 3 covering 13467374023 usec to 14361763671 usec
got 56 annotations on call # 4 cover

In [7]:
wagon.pull_annotations()

got 100 annotations on call # 1 covering 78122185209 usec to 447161111585 usec
got 100 annotations on call # 2 covering 447990693617 usec to 614647121216 usec
got 100 annotations on call # 3 covering 614700777466 usec to 660257408088 usec
got 100 annotations on call # 4 covering 660265902229 usec to 662061730354 usec
got 55 annotations on call # 5 covering 662088406135 usec to 717732786018 usec


In [8]:
wagon.filter_seizure_annotations()

Filtered       8.95% of all annotations


/users/wojemann/iEEG_processing/pioneer.py:199: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_annotations['IEEG Time'] = t_sec


In [9]:
# wagon.annotations.to_csv('example_annotations.csv')
wagon.seizure_annotations

,Duration,Description,IEEG Time
Time,,,
1970-01-01 21:42:02.185209,0 days,interictal,78122.185209
1970-01-01 23:30:05.083647,0 days,Patient Event,84605.083647
1970-01-02 01:53:58.610991,0 days,INTERICTAL,93238.610991
1970-01-02 03:32:56.624663,0 days,INTERICTAL,99176.624663
1970-01-02 05:23:58.602202,0 days,INTERICTAL,105838.602202
1970-01-02 07:36:09.667631,0 days,INTERICTAL,113769.667631
1970-01-02 09:09:48.647123,0 days,INTERICTAL,119388.647123
1970-01-02 10:58:50.580960,0 days,EVENT #1,125930.580960
1970-01-02 11:03:39.708890,0 days,Patient Event,126219.708890


#### Goal of next section is to make a csv containing all of the seizure times for each of these patients. The csv should contain HUPID, EEC, UEO, flag for stim or not, 